# Part I. ETL Pipeline for Pre-Processing the Files

## The following code combines all the event_data into a single csv file

#### Import Python packages 

In [129]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import timeit
import shutil

#### Creating list of filepaths to process original event csv data files

In [132]:
# Define function to create list of all csv files in a specified path
def get_files(filepath):
    file_path_list = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.csv'))
        for f in files :
            file_path_list.append(os.path.abspath(f))
    
    return file_path_list

# Go to main directory
os.chdir("..")

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# remove unwanted csv file in cache folder
if os.path.exists(filepath + '/.ipynb_checkpoints'):
    shutil.rmtree(filepath + '/.ipynb_checkpoints') 

# Get get of all files
file_path_list = get_files(filepath)    

print('{} csv files processed from {}'.format(len(file_path_list), filepath))
#print(file_path_list)
# Go back to notebooks directory
os.chdir("notebooks")

30 csv files processed from C:\Users\ericr\Google Drive\Online Courses\Udacity\Data Engineering\Part 1 - Data Modeling\data-modeling-nosql/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [38]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('../event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): # skip rows with no songplay info
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# Close the file object         
f.close()

#### Sort the CSV file by sessionId (9th col) and itemInSession (4th col) for easier exploration of the data

In [39]:
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('../event_datafile_new.csv', 'r', encoding='utf8', newline='') as f, \
open('../event_datafile_sorted.csv', 'w', encoding='utf8', newline='') as final:
    writer = csv.writer(final, delimiter=',', dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    reader = csv.reader(f, delimiter=',')
    next(reader)  # skip header
    sorted2 = sorted(reader, key=lambda row: (int(row[8]), int(row[3])))        
    for row in sorted2:
        writer.writerow(row)
        
# Close the file objects         
f.close()
final.close()

In [67]:
# check the number of rows in your csv file
with open('../event_datafile_sorted.csv', 'r', encoding = 'utf8') as f:
    print('{} rows in the combined csv file including header'.format(sum(1 for line in f)))
         
f.close()

6821 rows in the combined csv file including header


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data looks like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="../images/image_event_datafile_new.jpg">

#### Create a Cluster (Cassandra database connection)

In [77]:
# Make a connection to a Cassandra instance on your local machine 
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# 127.0.0.1 is the local cluster seen in the cqlsh shell window at startup
cluster = Cluster(contact_points=['127.0.0.1'], \
                  auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')) 

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [78]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace  
#### NOTE: Used the hints in this [Github post](https://gist.github.com/gioper86/b08b72d77c4e0aefa0137fc3655488dd) to modify the session parameters to allow a Pandas dataframe to be returned from a Cassandra query

In [79]:
# Function to allow a pandas DF be returned from a cassandra query
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

try:
    session.set_keyspace('udacity')
    session.row_factory = pandas_factory
    #session.default_fetch_size = 10000000 #needed for large queries, else driver will do pagination. Default is 50000.
except Exception as e:
    print(e)

## Create tables modeled  to run the following 3 queries

In [127]:
# Drop all tables first
session.execute("DROP TABLE IF EXISTS music_info_by_session")
session.execute("DROP TABLE IF EXISTS event_info_by_session")
session.execute("DROP TABLE IF EXISTS user_info_by_song")

# Create table for query 1
# PRIMARY KEY:
# Partition: sessionId. Clustering columns: itemInSession
# REASONING: sessionId and itemInSession should uniquely identify each music event.
# It is possible for the user to listen to the same song multiple times in a session.
# So to preserve that info, song and artist should NOT be included in the Primary Key.
query = "CREATE TABLE IF NOT EXISTS music_info_by_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, \
                  PRIMARY KEY (session_id, item_in_session))"
session.execute(query)

# Create table for query 2
# PRIMARY KEY:
# Partition: sessionId. Clustering columns: itemInSession
# REASONING: sessionId and itemInSession should uniquely identify each music event.
# UserId NOT NEEDED in PK because each sessionId is from 1 user
# itemInSession as Clustering column to get desired sorting
query = "CREATE TABLE IF NOT EXISTS event_info_by_session "
query = query + "(session_id int, item_in_session int, artist text, song text, first_name text, last_name text, \
                  user_id int, PRIMARY KEY (session_id, item_in_session))"
session.execute(query)

# Create table for query 3
# PRIMARY KEY:
# Partition: song. Clustering columns: user_id
# REASONING: This PK setup will get each unique song-user combo which will answer the question.
# NOTE: If a user listened to the same song more than once, the table will only 
# show info for ONE songplay, but that's okay, as we will still answer the question
query = "CREATE TABLE IF NOT EXISTS user_info_by_song "
query = query + "(song text, user_id int, first_name text, last_name text, \
                  PRIMARY KEY (song, user_id))"
session.execute(query)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1 (Table name: <font color=red>music_info_by_session</font>)
### Insert data into the 1st table and verify the query works as expected    

In [128]:
%%time
file = '../event_datafile_new.csv'

# Open file object
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert statement for 1st table (session_id is 9th column in CSV, so line[8], and so on)
        #print(float(line[5]))
        query = "INSERT INTO music_info_by_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

# Close the file object         
f.close()

Wall time: 33.8 s


#### Verify the data has been inserted by printing the first 10 rows and seeing the row count
#### WARNING: Using COUNT on a cassandra table would otherwise be dangerous in a multi-node database!!!

In [97]:
query = "SELECT COUNT(*) FROM music_info_by_session"
rows = session.execute(query)
df = rows._current_rows
print(df)

query = "SELECT * FROM music_info_by_session LIMIT 10"
rows = session.execute(query)
df = rows._current_rows
df

   count
0   6820


,sessionid,iteminsession,artist,length,song
0,23,0,Regina Spektor,191.085266,The Calculation (Album Version)
1,23,1,Octopus Project,250.957916,All Of The Champs That Ever Lived
2,23,2,Tegan And Sara,180.061584,So Jealous
3,23,3,Dragonette,153.390564,Okay Dolores
4,23,4,Lil Wayne / Eminem,229.589752,Drop The World
5,23,5,Soulja Boy Tell'em,201.116287,Let Me Get Em
6,23,6,Bodo Wartke,645.276306,Liebeslied (Sprachen: Deutsch_ Englisch_ Franz...
7,23,7,Evanescence,237.113022,Bring Me To Life
8,23,8,Van Halen,243.173416,Good Enough
9,23,9,The Academy Is...,209.762817,Paper Chase (Album Version)


### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [98]:
query = """SELECT artist, song, length FROM music_info_by_session \
        WHERE session_id = 338 AND item_in_session = 4"""

rows = session.execute(query)
df = rows._current_rows
df

,sessionid,iteminsession,artist,length,song
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


## Query 2 (Table name: <font color=red>event_info_by_session</font>)
### Insert data into the 2nd table and verify the query works as expected    

In [101]:
file = '../event_datafile_new.csv'

# Open file object
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert statement for 1st table (session_id is 9th column in CSV, so line[8], and so on)
        query = "INSERT INTO event_info_by_session (session_id, item_in_session, artist, song, \
                first_name, last_name, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], line[1], line[4], int(line[10])))
     
# Close the file object         
f.close()             

#### Verify the data has been inserted by printing the first 10 rows and seeing the row count
#### WARNING: Using COUNT ALL on a cassandra table would otherwise be dangerous in a multi-node database!!!

In [102]:
query = "SELECT COUNT(*) FROM event_info_by_session"
rows = session.execute(query)
df = rows._current_rows
print(df)

query = "SELECT * FROM event_info_by_session LIMIT 10"
rows = session.execute(query)
df = rows._current_rows
df                  

   count
0   6820


,session_id,item_in_session,artist,first_name,last_name,song,user_id
0,23,0,Regina Spektor,Layla,Griffin,The Calculation (Album Version),24
1,23,1,Octopus Project,Layla,Griffin,All Of The Champs That Ever Lived,24
2,23,2,Tegan And Sara,Layla,Griffin,So Jealous,24
3,23,3,Dragonette,Layla,Griffin,Okay Dolores,24
4,23,4,Lil Wayne / Eminem,Layla,Griffin,Drop The World,24
5,23,5,Soulja Boy Tell'em,Layla,Griffin,Let Me Get Em,24
6,23,6,Bodo Wartke,Layla,Griffin,Liebeslied (Sprachen: Deutsch_ Englisch_ Franz...,24
7,23,7,Evanescence,Layla,Griffin,Bring Me To Life,24
8,23,8,Van Halen,Layla,Griffin,Good Enough,24
9,23,9,The Academy Is...,Layla,Griffin,Paper Chase (Album Version),24


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [107]:
query = """SELECT artist, song, first_name, last_name \
            FROM event_info_by_session \
            WHERE session_id = 182"""

rows = session.execute(query)
df = rows._current_rows
df

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3 (Table name: <font color=red>user_info_by_song</font>)
### Insert data into the 3rd table and verify the query works as expected    

In [109]:
file = '../event_datafile_new.csv'

# Open file object
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert statement for 1st table (session_id is 9th column in CSV, so line[8], and so on)
        query = "INSERT INTO user_info_by_song (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
     
# Close the file object         
f.close()    

#### Verify the data has been inserted by printing the first 10 rows and seeing the row count
#### WARNING: Using COUNT ALL on a cassandra table would otherwise be dangerous in a multi-node database!!!

In [110]:
query = "SELECT COUNT(*) FROM user_info_by_song"
rows = session.execute(query)
df = rows._current_rows
print(df)

query = "SELECT * FROM user_info_by_song LIMIT 10"
rows = session.execute(query)
df = rows._current_rows
df     

   count
0   6618


,song,user_id,first_name,last_name
0,Wonder What's Next,49,Chloe,Cuevas
1,In The Dragon's Den,49,Chloe,Cuevas
2,Too Tough (1994 Digital Remaster),44,Aleena,Kirby
3,Rio De Janeiro Blue (Album Version),49,Chloe,Cuevas
4,My Place,15,Lily,Koch
5,My Place,73,Jacob,Klein
6,The Lucky Ones,24,Layla,Griffin
7,I Want You Now,80,Tegan,Levine
8,Why Worry,88,Mohammad,Rodriguez
9,TvÃÂ¡rÃÂ­ v TvÃÂ¡r,97,Kate,Harrell


#### NOTE: there are 202 less rows in this table, because 202 events were repeat songplays by the same user. It is okay these repeat songplays have been removed, because they are not relevant to the question.

### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [113]:
query = """SELECT first_name, last_name \
            FROM user_info_by_song \
            WHERE song = 'All Hands Against His Own'"""

rows = session.execute(query)
df = rows._current_rows
df

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [4]:
session.execute("DROP TABLE IF EXISTS music_info_by_session")
session.execute("DROP TABLE IF EXISTS event_info_by_session")
session.execute("DROP TABLE IF EXISTS user_info_by_song")

### Close the session and cluster connection¶

In [133]:
session.shutdown()
cluster.shutdown()